## Look Ahead K-Means

In [1]:
import time
import tracemalloc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris, load_wine, make_blobs
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from IPython.display import display
import ipywidgets as widgets

In [2]:

# -----------------------------
# CONFIGURATION
# -----------------------------
CONFIG = {
    'n_candidates': 20,
    'rollout_depth': 6,
    'n_iterations': 10,
    'random_seed': 123,
    'datasets': {
        'Iris': (load_iris().data, 3),
        'Wine': (load_wine().data, 3),
        'Overlapping': (make_blobs(n_samples=500, centers=[[0, 0], [3, 3]],
                                     cluster_std=1.5, random_state=101)[0] @ np.array([[0.6, -0.6], [0.4, 0.8]]), 2),
        'Noisy': (np.hstack([
            make_blobs(n_samples=500, centers=3, cluster_std=1.2, random_state=2024)[0],
            np.random.normal(0, 5, (500, 5))]), 3)
    }
}

# -----------------------------
# Lookahead Initialization
# -----------------------------
def lookahead_init(X, k, n_candidates, rollout_depth, seed):
    best_score = -np.inf
    best_init = None
    rng = np.random.default_rng(seed)
    for _ in range(n_candidates):
        centers = X[rng.choice(len(X), size=k, replace=False)]
        for _ in range(rollout_depth):
            labels = np.argmin(np.linalg.norm(X[:, None] - centers[None, :], axis=2), axis=1)
            centers = np.array([
                X[labels == j].mean(axis=0) if np.any(labels == j) else centers[j]
                for j in range(k)
            ])
        score = silhouette_score(X, labels)
        if score > best_score:
            best_score = score
            best_init = centers
    return best_init

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate_kmeans_with_history(X, k, use_lookahead, config):
    tracemalloc.start()
    start = time.time()

    if use_lookahead:
        init_centers = lookahead_init(X, k, config['n_candidates'], config['rollout_depth'], config['random_seed'])
        centers = init_centers.copy()
    else:
        model = KMeans(n_clusters=k, init='k-means++', n_init=1, max_iter=1, random_state=config['random_seed'])
        model.fit(X)
        centers = model.cluster_centers_.copy()

    history = []
    for _ in range(config['n_iterations']):
        distances = np.linalg.norm(X[:, None] - centers[None, :], axis=2)
        labels = np.argmin(distances, axis=1)
        score = silhouette_score(X, labels)
        history.append(round(score, 2))
        centers = np.array([
            X[labels == i].mean(axis=0) if np.any(labels == i) else centers[i] for i in range(k)
        ])

    runtime = round(time.time() - start, 2)
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    peak_memory_mb = round(peak / (1024 * 1024), 2)

    return history, runtime, peak_memory_mb

In [3]:
# -----------------------------
# Main Experiment Loop
# -----------------------------
histories = []
summary = []

for name, (X, k) in CONFIG['datasets'].items():
    std_hist, std_runtime, std_mem = evaluate_kmeans_with_history(X, k, False, CONFIG)
    la_hist, la_runtime, la_mem = evaluate_kmeans_with_history(X, k, True, CONFIG)

    for i, (s, l) in enumerate(zip(std_hist, la_hist), start=1):
        histories.append([name, i, s, l, std_runtime, la_runtime, std_mem, la_mem])

    summary.append([
        name,
        round(np.mean(std_hist), 2), round(np.mean(la_hist), 2),
        std_runtime, la_runtime,
        std_mem, la_mem
    ])

In [4]:
# -----------------------------
# Output Tables
# -----------------------------
history_df = pd.DataFrame(histories, columns=[
    'Dataset', 'Iteration', 'Standard Init', 'Lookahead Init',
    'Standard Time (s)', 'Lookahead Time (s)', 'Standard Mem (MB)', 'Lookahead Mem (MB)'
])
print("Per-Iteration Results:")
display(widgets.VBox([widgets.Output(layout={'overflow_x': 'auto', 'border': '1px solid gray', 'max_height': '300px'})]))
display(history_df)

summary_df = pd.DataFrame(summary, columns=[
    'Dataset', 'Mean Std Silhouette', 'Mean LA Silhouette',
    'Std Time (s)', 'LA Time (s)', 'Std Mem (MB)', 'LA Mem (MB)'
])
print("\nSummary Results:")
display(widgets.VBox([widgets.Output(layout={'overflow_x': 'auto', 'border': '1px solid gray', 'max_height': '200px'})]))
display(summary_df)

Per-Iteration Results:


,Dataset,Iteration,Standard Init,Lookahead Init,Standard Time (s),Lookahead Time (s),Standard Mem (MB),Lookahead Mem (MB)
0,Iris,1,0.55,0.55,0.25,0.16,0.60,0.36
1,Iris,2,0.55,0.55,0.25,0.16,0.60,0.36
2,Iris,3,0.55,0.55,0.25,0.16,0.60,0.36
3,Iris,4,0.55,0.55,0.25,0.16,0.60,0.36
4,Iris,5,0.55,0.55,0.25,0.16,0.60,0.36
5,Iris,6,0.55,0.55,0.25,0.16,0.60,0.36
6,Iris,7,0.55,0.55,0.25,0.16,0.60,0.36
7,Iris,8,0.55,0.55,0.25,0.16,0.60,0.36
8,Iris,9,0.55,0.55,0.25,0.16,0.60,0.36
9,Iris,10,0.55,0.55,0.25,0.16,0.60,0.36



Summary Results:


,Dataset,Mean Std Silhouette,Mean LA Silhouette,Std Time (s),LA Time (s),Std Mem (MB),LA Mem (MB)
0,Iris,0.55,0.55,0.25,0.16,0.60,0.36
1,Wine,0.57,0.57,0.08,0.12,0.50,0.50
2,Overlapping,0.39,0.39,0.08,0.24,2.00,2.00
3,Noisy,0.18,0.22,0.08,0.26,2.01,2.01
